In [1]:
import tensorflow as tf
import numpy as np
import keras_tuner as kt
import pandas as pd
import seq_generator
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds
import json
import os
import tqdm

pars = json.load(open("tuner_run/best_hps.json"))

def build_cnn():

    inputs = tf.keras.Input([260,260,3])

    img_augmentation = tf.keras.models.Sequential(
        [
            tf.keras.layers.RandomRotation(factor=0.15),
            tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
            tf.keras.layers.RandomContrast(factor=0.1),
        ],
        name="img_augmentation",
    )

    x = tf.keras.layers.Resizing(224,224) (inputs)

    x = img_augmentation(x)

    base = tf.keras.applications.EfficientNetB0(
                include_top=False,
                weights='imagenet',
                input_tensor= x)

    base.trainable = False

    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(base.output)

    return tf.keras.Model(inputs,x)

def build_model(hps,cnn):


    inputs = tf.keras.Input([5,260,260,3])
    x = layers.TimeDistributed(cnn)(inputs)

    drop1 = hps["drop1"]

    x = layers.TimeDistributed( layers.Dropout(drop1), name = "drop1") (x)

    hidden_units =  hps["hidden_units"]

    if hps["rnn"] == "lstm":
        x = layers.LSTM(hidden_units, name = "lstm")(x)
    else:
        x = layers.GRU(hidden_units, name = "gru") (x)

    drop2 = hps["drop2"]
    x = layers.Dropout(drop2,name ="drop2")(x)

    outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred",kernel_regularizer=tf.keras.regularizers.L2())(x)

    learning_rate = hps["lr"]

    model = tf.keras.Model(inputs,outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

def load_model(i):
    cnn = build_cnn()
    model = build_model(pars,cnn)
    name = f"b0_ensemple_{i}"
    model.load_weights(f"tuner_run/ensemble/weights/{name}_warm/cp.cpkt")
    return model

_ , val_ds = seq_generator.get_train_and_val(batch_size=20)
val_ds = val_ds.map(lambda x,y: (x,tf.one_hot(y,2)))

c:\Users\ual\.conda\envs\felix\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# losses = {}
# cnn = build_cnn()
# model = build_model(pars,cnn)
# for i in tqdm.tqdm(range(21)): 

#     name = f"b0_ensemple_{i}"
#     model.load_weights(f"tuner_run/ensemble/weights/{name}_warm/cp.cpkt")
#     _, acc = model.evaluate(val_ds,verbose=0)
    
#     losses[i]= acc

#     with open("tmp/new_results.json","w") as f:
#         json.dump(losses,f)

In [3]:
with open("tmp/new_results.json","r") as f:
    losses = json.load(f)
losses = np.array(list(losses.values()))
best = np.argsort(-losses)
losses[best]
best

array([20,  4,  2,  1, 16, 17,  5,  3, 18, 10, 19,  8, 14, 11, 15, 12,  7,
        9,  0,  6, 13], dtype=int64)

In [3]:
# ensemble of 3
inputs = tf.keras.Input([5,260,260,3])
x = [load_model(i)(inputs) for i in best[:3]]

outputs = tf.math.reduce_mean(x,axis = 0)

ensemble = tf.keras.Model(inputs,outputs)
ensemble.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
ensemble.evaluate(val_ds)

225/225 [==============================] - 47s 157ms/step - loss: 0.1829 - accuracy: 0.9513


[0.1828595995903015, 0.9513117074966431]

In [4]:
# ensemble of 5
inputs = tf.keras.Input([5,260,260,3])
x = [load_model(i)(inputs) for i in best[:5]]

outputs = tf.math.reduce_mean(x,axis = 0)

ensemble = tf.keras.Model(inputs,outputs)
ensemble.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
ensemble.evaluate(val_ds)

225/225 [==============================] - 72s 256ms/step - loss: 0.1861 - accuracy: 0.9533


[0.18611377477645874, 0.9533125758171082]

In [6]:
# ensemble of 5
inputs = tf.keras.Input([5,260,260,3])
x = [load_model(i)(inputs) for i in range(5)]

outputs = tf.math.reduce_mean(x,axis = 0)

ensemble = tf.keras.Model(inputs,outputs)
ensemble.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
ensemble.evaluate(val_ds)

225/225 [==============================] - 73s 256ms/step - loss: 0.1952 - accuracy: 0.9511


[0.19520433247089386, 0.9510893821716309]